In [ ]:
import json
import pandas
import time
import os
import csv
from functools import reduce

In [ ]:
inputDir = '..//scrapedData//'
outputDir = '..//summarizedData//'

## Load the id config
idConfig = json.loads(open('..//config//idConfig.json', "r").read())

## Load the source file dict
sourceFiles = json.loads(open('..//config//sourcefiles.json', "r").read())

In [ ]:
## If there are multiple files for any given dataset, then this function will combine all of those
## records into a single dicti
def mergeSourceFiles (source, outputDir, sourceFiles):
    recordList = []
    for file in sourceFiles[source]:  
        file = json.loads(open(outputDir + file, "r", encoding="utf-8").read())
        for record in file:
            recordList.append(record)
    return recordList

In [ ]:
allConfData = mergeSourceFiles('allConf', inputDir, sourceFiles)

In [ ]:
## Clean up dirty names
def mungeID(playerString):
    return ''.join(e for e in playerString if e.isalnum()).lower().replace("jr.", "").replace("st.", "state")  

In [ ]:
def createNewID (fieldList, thisDict, fieldAgg):
    finalID= ''
    for i in thisDict:
        for idx, val in enumerate(fieldList):
            if (type(i[val]) is list):
                i[val]= mungeID(i[val][0])
                if (len(fieldList) -1 == idx):
                    finalID += str(i[val]).strip('[]').strip("''")
            elif (type(val) is not list):
                i[val] = mungeID(i[val])
                if (len(fieldList) - 1 == idx):
                    finalID += i[val]
                else:
                    finalID = i[val] + fieldAgg
        i['ID'] = finalID
        finalID=''

In [ ]:
def search(name, team, dataList):
    return [element for element in dataList if (element['playerName'] == name and element['school'] == team)]

def searchID(identifier, dataList):
    return [element for element in dataList if (element['ID'] == identifier)]


In [ ]:
createNewID(idConfig['allConf'], allConfData, '_')

In [ ]:
finalOutput = []
for x in allConfData:
    if (len(searchID(x['ID'], finalOutput)) == 0):
        try:
            playerList = search(x['playerName'], x['school'], allConfData)
            finalPlayer = {}
            finalPlayer['ID'] = x['ID']
            finalPlayer['playerName'] = x['playerName']
            finalPlayer['school'] = x['school']
            finalPlayer['team'] = x['team']
            year = 2021
            coaches = media = []
        
        except:
            print (x)
        
        finalOutput.append(finalPlayer)

{'Year': '2013', 'POSITION': 'RS', 'First Name': 'Janarion', 'Last Name': 'Grant', 'Team': '2', 'playerName': 'janariongrant', 'school': 'rutgers', 'ID': 'janariongrant_rutgers'}
{'Year': '2013', 'POSITION': 'WR', 'First Name': 'Stefon', 'Last Name': 'Diggs', 'Team': '1', 'playerName': 'stefondiggs', 'school': 'maryland', 'ID': 'stefondiggs_maryland'}
{'Year': '2013', 'POSITION': 'KR', 'First Name': 'Stefon', 'Last Name': 'Diggs', 'Team': '2', 'playerName': 'stefondiggs', 'school': 'maryland', 'ID': 'stefondiggs_maryland'}
{'Year': '2012', 'POSITION': 'QB', 'First Name': 'Geno', 'Last Name': 'Smith', 'Team': '2', 'playerName': 'genosmith', 'school': 'westvirginia', 'ID': 'genosmith_westvirginia'}
{'Year': '2010', 'POSITION': 'OL', 'First Name': 'Joe', 'Last Name': 'Madsen', 'Team': '1', 'playerName': 'joemadsen', 'school': 'westvirginia', 'ID': 'joemadsen_westvirginia'}
{'Year': '2011', 'POSITION': 'OL', 'First Name': 'Blaize', 'Last Name': 'Foltz', 'Team': '1', 'playerName': 'blaizefo

ValueError: min() arg is an empty sequence

In [ ]:
print(finalOutput)

[{'ID': 'drewlock_missouri', 'playerName': 'drewlock', 'school': 'missouri', 'team': '1'}, {'ID': 'jarrettstidham_auburn', 'playerName': 'jarrettstidham', 'school': 'auburn', 'team': '1'}, {'ID': 'kerryonjohnson_auburn', 'playerName': 'kerryonjohnson', 'school': 'auburn', 'team': '1'}, {'ID': 'bennysnelljr_kentucky', 'playerName': 'bennysnelljr', 'school': 'kentucky', 'team': '2'}, {'ID': 'nickchubb_georgia', 'playerName': 'nickchubb', 'school': 'georgia', 'team': '2'}, {'ID': 'derriusguice_lsu', 'playerName': 'derriusguice', 'school': 'lsu', 'team': '2'}, {'ID': 'ajbrown_olemiss', 'playerName': 'ajbrown', 'school': 'olemiss', 'team': '1'}, {'ID': 'calvinridley_alabama', 'playerName': 'calvinridley', 'school': 'alabama', 'team': '1'}, {'ID': 'jmonmoore_missouri', 'playerName': 'jmonmoore', 'school': 'missouri', 'team': '2'}, {'ID': 'christiankirk_texasam', 'playerName': 'christiankirk', 'school': 'texasam', 'team': '2'}, {'ID': 'willclapp_lsu', 'playerName': 'willclapp', 'school': 'lsu

In [ ]:
##Delete the columns I no longer care about after creating the ID
##This list will need to expand after setting up blockers and truly linking
for record in finalOutput:
    del record['playerName']
    del record['school']


In [ ]:
with open(outputDir + "allConf.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(finalOutput))